# See predicted function

In [62]:
import matlab.engine
from optimize import *
from skopt.acquisition import _gaussian_acquisition
result = load("./optims/scenario9/forest_0_400.opt")

In [71]:
N = 20

# K - number of axes
# N - number of eval points per axis

K = result.space.n_dims
if(isinstance(N, int)):
    N = (N,) * K

In [72]:
# ground-truth
truth = {'names': [],
         'priors': [],
         'x': np.array(result.x_iters),
         'y': result.func_vals,
         'best': np.where(result.func_vals == result.fun)[0][0] + 1}    # matlab indexing

# axes names and values
axes = {}

for idx in range(K):
    dim = result.space.dimensions[idx]
    
    truth['names'].append(dim.name)
    if(dim.prior == 'log-uniform'):
        truth['priors'].append('log')
        func = np.geomspace
    else:
        truth['priors'].append('uniform')
        func = np.linspace
    axes[dim.name] = func(dim.low, dim.high, num=N[idx])

In [116]:
# surrogate model
surrogate = {'mean': np.zeros(N), 'std': np.zeros(N)}

# variable number of for-loops -> recursion
def iterate(i, idx, xs, idxs):
    if(i >= K):
        # base case - evaluate
        x = []
        for j in range(K):
            x.append(axes[truth['names'][j]][idx[j]])
        idx2 = tuple(idx)
        
        # true value and index
        xs.append(x)
        idxs.append(idx2)
        return
    for val in range(N[i]):
        # recursion - proceed to next dimension
        idx.append(val)
        iterate(i+1, idx, xs, idxs)
        idx.pop()

In [117]:
xs = []
idxs = []
iterate(0, [], xs, idxs)

value = result.models[-1].predict(result.space.transform(xs), return_std=True)

for i in range(len(idxs)):
    surrogate['mean'][idxs[i]] = value[0][i]
    surrogate['std'][idxs[i]] = value[1][i]

In [119]:
idxs

[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (0, 5),
 (0, 6),
 (0, 7),
 (0, 8),
 (0, 9),
 (0, 10),
 (0, 11),
 (0, 12),
 (0, 13),
 (0, 14),
 (0, 15),
 (0, 16),
 (0, 17),
 (0, 18),
 (0, 19),
 (1, 0),
 (1, 1),
 (1, 2),
 (1, 3),
 (1, 4),
 (1, 5),
 (1, 6),
 (1, 7),
 (1, 8),
 (1, 9),
 (1, 10),
 (1, 11),
 (1, 12),
 (1, 13),
 (1, 14),
 (1, 15),
 (1, 16),
 (1, 17),
 (1, 18),
 (1, 19),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 3),
 (2, 4),
 (2, 5),
 (2, 6),
 (2, 7),
 (2, 8),
 (2, 9),
 (2, 10),
 (2, 11),
 (2, 12),
 (2, 13),
 (2, 14),
 (2, 15),
 (2, 16),
 (2, 17),
 (2, 18),
 (2, 19),
 (3, 0),
 (3, 1),
 (3, 2),
 (3, 3),
 (3, 4),
 (3, 5),
 (3, 6),
 (3, 7),
 (3, 8),
 (3, 9),
 (3, 10),
 (3, 11),
 (3, 12),
 (3, 13),
 (3, 14),
 (3, 15),
 (3, 16),
 (3, 17),
 (3, 18),
 (3, 19),
 (4, 0),
 (4, 1),
 (4, 2),
 (4, 3),
 (4, 4),
 (4, 5),
 (4, 6),
 (4, 7),
 (4, 8),
 (4, 9),
 (4, 10),
 (4, 11),
 (4, 12),
 (4, 13),
 (4, 14),
 (4, 15),
 (4, 16),
 (4, 17),
 (4, 18),
 (4, 19),
 (5, 0),
 (5, 1),
 (5, 2),
 (5, 3),
 (5, 4),
 (5, 

In [113]:
value

(array([0.87726274, 0.8835544 , 0.90197185, 0.92343742, 0.92773941,
        0.92158231, 0.94366523, 0.93816464, 0.93834458, 0.94359897,
        0.92998425, 0.92530479, 0.89129605, 0.86818635, 0.87154162,
        0.86221648, 0.83506362, 0.82471419, 0.80063902, 0.77034389,
        0.87628438, 0.8823434 , 0.90076085, 0.92222642, 0.92652841,
        0.92049969, 0.94281525, 0.93710863, 0.93619631, 0.94109237,
        0.92653183, 0.92185238, 0.88784364, 0.860143  , 0.86278577,
        0.85346063, 0.82012113, 0.80328976, 0.77787132, 0.74031924,
        0.83315208, 0.83921111, 0.86535216, 0.88453427, 0.8766269 ,
        0.86071348, 0.87719836, 0.85455207, 0.85904127, 0.85963674,
        0.85955905, 0.84027391, 0.81249361, 0.80210327, 0.80737767,
        0.79528306, 0.76803167, 0.74842   , 0.69714529, 0.6607173 ,
        0.56758015, 0.57363918, 0.59293234, 0.61827405, 0.57906893,
        0.55912602, 0.56862908, 0.57773698, 0.56958867, 0.54722477,
        0.51878115, 0.50264495, 0.49113829, 0.47

In [114]:
savemat('./figures/hyper_opt.mat', mdict={'surrogate':surrogate, 'axxes':axes, 'truth':truth}, oned_as='column')

C:\Program Files\Anaconda3\envs\tensorflow\lib\site-packages\scipy\io\matlab\miobase.py:414: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  arr[empties] = ' '
